<a href="https://colab.research.google.com/github/chw8207/fastai_pytorch/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D_fastai_%EA%B0%9C%EC%9A%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from fastai.vision.all import *

### GPU 설정

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0' :
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

Found GPU at: /device:GPU:0


In [3]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 364817688102640323
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 40129593344
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12890420426624130047
physical_device_desc: "device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0"
xla_global_id: 416903419
]


### 이미지 인식 모델

In [4]:
path = untar_data(URLs.PETS)/'images'

In [5]:
# 데이터셋에 포함된 파일명 규칙에 따라
# 고양이 레이블을 지정하는 함수(고양이(대문자) : True)
def is_cat(x) :
  return x[0].isupper()

In [14]:
# fastai에 데이터셋의 종류와 구조화 방법 알려주기
# from_name_func : 파일명에 적용할 함수로 레이블 추출
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2,
    seed=42, label_func=is_cat, item_tfms=Resize(224)
)

In [15]:
# 합성곱 신경상 생성
learn = cnn_learner(dls, resnet34, metrics=error_rate)

/usr/local/lib/python3.10/dist-packages/fastai/vision/learner.py:288: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [16]:
# 모델 적합
learn.fine_tune(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.177777,0.013184,0.004060,00:17


epoch,train_loss,valid_loss,error_rate,time
0,0.058158,0.027070,0.004060,00:11
